In [ ]:
import duckdb
import pandas as pd
from statsbombpy import sb

# DuckDB bağlantısı
#con = duckdb.connect("bitirme.db")


In [ ]:
def pass_data_creation(events_extra_info,match_ID):
    
    events_df = sb.events(match_id=match_ID)
    events_df = events_df.set_index("match_id").join(
        events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
    ).reset_index()

    # Koordinatları ayır
    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

    # Paslar tablosunu oluştur
    passes = events_df[(events_df.type == "Pass")]
    passes = passes[passes["pass_outcome"] != "Injury Clearance"]
    shots = events_df[(events_df.type == "Shot")][["player","shot_type","shot_key_pass_id","shot_outcome","shot_statsbomb_xg"]].dropna()
    passes_M = passes.set_index("id").join(
        shots.set_index("shot_key_pass_id"), how="left", rsuffix='_shotFromPass'
    )
    passes_M = passes_M[passes_M["pass_outcome"] != "Injury Clearance"]

    passes_M["pass_succes"] = passes_M.pass_outcome.isna()
    passes_M["f3rd_pass"] = (passes_M.x < 80) & (passes_M.pass_end_x > 80)
    passes_M["pass_progression"] = passes_M["pass_end_x"] - passes_M["x"]
    passes_M.loc[passes_M.pass_outcome.notna(), "pass_progression"] = None
    passes_M["shot_assisted_pass"] = (passes_M["pass_succes"] == True) &  (passes_M["shot_outcome_shotFromPass"].notna())
    passes_M["goal_assisted_pass"] = (passes_M["pass_succes"] == True) & (passes_M["shot_outcome_shotFromPass"] == "Goal")
    
    passes_M["cross_pass"] = (passes_M.pass_cross == True)
    passes_M["box_pass"] = (
        ((passes_M.x < 102) | ((passes_M.y < 18) | (passes_M.y > 62))) & ((passes_M.pass_end_x > 102) & ((passes_M.pass_end_y > 18) & (passes_M.pass_end_y < 62)))
    )

    passes_M["rowID"] = passes_M.index

    return passes_M

In [ ]:
def foul_data_creation(events_extra_info,match_ID):

    events_df = sb.events(match_id=match_ID)
    events_df = events_df.set_index("match_id").join(
        events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
    ).reset_index()

    # Koordinatları ayır
    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

    # Fauller tablosunu oluştur

    # Set of existing columns in the DataFrame
    existing_columns = set(events_df.columns)

    # Set of columns you want to check
    columns_to_check = {"foul_committed_advantage", "foul_committed_card"}

    # Find which columns exist and which do not
    missing = columns_to_check - existing_columns
    if len(missing) == 0:
        foul_commited = events_df[(events_df.type == "Foul Committed")][
        ["id", "type", "player", "position","team", "related_events", "foul_committed_advantage", "foul_committed_card", "x", "y"]
        ].set_index("id")

    elif len(missing) > 1:
        print("there was no commited advantage nor commited card in this match ")
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player","position","team", "related_events", "x", "y"]]
        foul_commited["foul_committed_advantage_foul_commitedJ"] = None
        foul_commited["foul_committed_card_foul_commitedJ"] = None
        foul_commited = foul_commited.set_index("id")
    
    elif "foul_committed_card" in missing:
        print("there was no commited card in this match ")
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player","position","team", "related_events", "foul_committed_advantage", "x", "y"]]
        foul_commited["foul_committed_card_foul_commitedJ"] = None
        foul_commited = foul_commited.set_index("id")
    
    elif "foul_committed_advantage" in missing:
        print("there was no commited advantage in this match ")
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player","position","team", "related_events", "foul_committed_card", "x", "y"]]
        foul_commited["foul_committed_advantage_foul_commitedJ"] = None
        foul_commited = foul_commited.set_index("id")

    
    fouls = events_df[(events_df.type == "Foul Won")].explode("related_events")
    fouls_M = fouls.set_index("related_events").join(foul_commited, how="left", rsuffix='_foul_commitedJ')
    fouls_M = fouls_M[fouls_M["player_foul_commitedJ"].notna()]
    fouls_M["rowID"] = fouls_M.index
    return fouls_M

In [ ]:
def event_data_creation(events_extra_info,match_ID):
    
    events_df = sb.events(match_id=match_ID)
    events_df.set_index("id")

    events_df['timestamp'] = pd.to_datetime(events_df['timestamp'], format='%H:%M:%S.%f')
    events_df.loc[events_df['minute'] >= 45, 'timestamp'] += pd.Timedelta(minutes=45)
    posessions_sorted = events_df[["duration","kick_off","minute","timestamp","possession","possession_team"]].sort_values("timestamp")
    possession_time = posessions_sorted.groupby("possession").apply(
        lambda x: (x["timestamp"].max() - x["timestamp"].min()).total_seconds()
    ).reset_index(name="possession_time")
    events_df = events_df.set_index("possession").join(possession_time.set_index("possession"),how="left").reset_index()

    players = (
        events_df[events_df["player"].notnull()][["player", "position", "team"]]
        .groupby("player")["position"]
        .agg(lambda x: x.value_counts().idxmax())
        .reset_index()
    )
    events_df = events_df.set_index("player").join(players.set_index("player"),how="left",rsuffix="_most_played").reset_index()

    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)
    passes = events_df[(events_df.type=="Pass")]
    passes = passes[passes["pass_outcome"] != "Injury Clearance"]
    events_df = events_df[((events_df.type=="Pass")&(events_df["pass_outcome"] == "Injury Clearance"))|(events_df.type!="Pass")]
    shots_fromPass = events_df[(events_df.type=="Shot")][["player","shot_type","shot_key_pass_id","shot_outcome","shot_statsbomb_xg"]]
    passes_M = passes.set_index("id").join(shots_fromPass.set_index("shot_key_pass_id"),how="left",rsuffix='_shotFromPass').reset_index()
    passes_M["pass_succes"] = passes_M.pass_outcome.isna()
    passes_M["f3rd_pass"] = (passes_M.x < 80) & (passes_M.pass_end_x > 80)
    passes_M["pass_progression"] = passes_M["x"] - passes_M["pass_end_x"]
    passes_M["pass_progression"].loc[passes_M.pass_outcome.notna()] = None
    passes_M["shot_assisted_pass"] = passes_M.pass_outcome.isna() &  passes_M["shot_outcome_shotFromPass"].notna()
    passes_M["goal_assisted_pass"] = passes_M.pass_outcome.isna() & passes_M["shot_outcome_shotFromPass"] == "Goal"
    passes_M["cross"] = (passes_M.pass_cross == True)
    passes_M["box_pass"] = (
        (passes_M.x < 102)
        & ((passes_M.y < 18) | (passes_M.y > 62))
        & (passes_M.pass_end_x > 102)
        & ((passes_M.pass_end_y > 18) | (passes_M.pass_end_y < 62))
    )
    passes_M = passes_M.set_index("pass_recipient").join(players.set_index("player"),how="left",rsuffix="_re_most_played").reset_index()

    foul_commited = events_df[(events_df.type=="Foul Committed")][["id","player","position","team","timestamp","minute","type","foul_committed_advantage","foul_committed_card","x","y"]]
    fouls = events_df[(events_df["type"]=="Foul Won")]
    events_df = events_df[~((events_df["type"]=="Foul Won") | (events_df["type"]=="Foul Committed"))]
    fouls = fouls.explode("related_events")
    fouls_M = fouls.set_index("related_events").join(foul_commited.set_index("id"),how="left", rsuffix='_foul_commitedJ').reset_index()
    fouls_M = fouls_M[fouls_M["player_foul_commitedJ"].notna()]

    events_df = pd.concat([events_df,passes_M,fouls_M])

    return events_df










In [ ]:
passDF = pd.DataFrame()
foulsDF= pd.DataFrame()


# # Lig ve sezon bilgileri
# competitions_and_seasons = [
#     (11, 27, "2015/16 La Liga"),
#     (9, 27, "2015/16 Bundesliga"),
#     (2, 27, "2015/16 Premier League"),
#     (12, 27, "2015/16 Serie A"),
#     (7, 27, "2015/16 Ligue 1"),
#     (43, 106, "FIFA World Cup 22"),
#     (43, 3, "FIFA World Cup 18"),
#     (55, 282, "Euro 2024"),
#     (55, 43, "Euro 2020")
# ]




laliga = (11, 27, "2015/16 La Liga")
bundesliga = (9, 27, "2015/16 Bundesliga")
premierleague = (2, 27, "2015/16 Premier League")
seriea = (12, 27, "2015/16 Serie A")
ligue1 = (7, 27, "2015/16 Ligue 1")
worldcup22 = (43, 106, "FIFA World Cup 22")
worldcup18 = (43, 3, "FIFA World Cup 18")
euru24 = (55, 282, "Euro 2024")
euro20 = (55, 43, "Euro 2020")

In [ ]:


#for competition, season, competitions_name in competitions_and_seasons:
    # Maçları al

competition, season, competitions_name = laliga
matches = sb.matches(competition_id=competition, season_id=season)

events_extra_info = matches[[
    "match_id",
    "match_date",
    "kick_off",
    "competition",
    "home_team",
    "away_team",
    "home_score",
    "away_score",
    "match_status",
    "match_week",
    "competition_stage",
    "referee",
    "home_managers",
    "away_managers",
]]

matchCount = matches.shape[0]
counter = 0


for match_ID in matches["match_id"].unique():
    counter += 1
    info = events_extra_info[events_extra_info["match_id"]==match_ID]
    print(f"{counter}/{matchCount} | league: {info['competition'].values[0]}, match id {match_ID}, {info['home_team'].values[0]}-{info['away_team'].values[0]}")
    passData = pass_data_creation(events_extra_info,match_ID)
    #foulData = foul_data_creation(events_extra_info,match_ID)
    passDF = pd.concat([passDF, passData], axis=0, ignore_index=True)
    #foulsDF = pd.concat([foulsDF, foulData], axis=0, ignore_index=True)


In [ ]:
columns = [
    "competition",
    "home_team",
    "away_team",
    "referee",
    "x",
    "y",
    "player",
    "position",
    "possession_team",
    "type_foul_commitedJ",
    "player_foul_commitedJ",
    "position_foul_commitedJ",
    "team_foul_commitedJ",
    "related_events",
    "foul_committed_advantage_foul_commitedJ",
    "x_foul_commitedJ",
    "y_foul_commitedJ",
    "foul_committed_card_foul_commitedJ",
    "rowID"
]


foulsDF[columns].head(10)

In [ ]:
passDF[[
    "match_id",
    "competition",
    "home_team",
    "away_team",
    "referee",
    "player",
    "x",
    "y",
    "pass_end_x",
    "pass_end_y",
    "player_shotFromPass",
    "shot_outcome_shotFromPass",
    "shot_statsbomb_xg_shotFromPass",
    "pass_succes",
    "f3rd_pass",
    "pass_progression",
    "shot_assisted_pass",
    "goal_assisted_pass",
    "box_pass"
]][passDF["shot_outcome_shotFromPass"]=="Goal"]

In [ ]:

# Connect to a DuckDB database (or create one if it doesn't exist)
# Use ':memory:' for an in-memory database or specify a filename for a persistent database
con = duckdb.connect("example.db")



In [ ]:
# Write the Pandas DataFrame to the DuckDB database
# This creates a table named 'my_table'
con.execute("CREATE TABLE passes AS SELECT * FROM passDF")

# You can also use the `from_df` function for direct insertion:
# con.from_df(df, "my_table")

# Verify the data was written to the database
result = con.execute("SELECT * FROM passes").fetchall()
print(result)

# Close the connection


In [ ]:
con.execute("DROP TABLE passes")


In [ ]:
# Close the connection
con.close()

